In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "X-Linked_Lymphoproliferative_Syndrome"
cohort = "GSE180394"

# Input paths
in_trait_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome"
in_cohort_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome/GSE180394"

# Output paths
out_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/GSE180394.csv"
out_gene_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE180394.csv"
out_clinical_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/clinical_data/GSE180394.csv"
json_path = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Yes, this dataset contains gene expression data as it mentions using microarrays to analyze the transcriptome
# and specifically states "Profiling was performed on Affymetrix ST2.1 microarray platform"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait: We can use the sample group from row 0 which indicates disease status
trait_row = 0

# For age: No age information is available in the sample characteristics
age_row = None

# For gender: No gender information is available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert sample group values to binary trait values (0 for healthy, 1 for disease)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Living donor is considered healthy (0), all others have disease (1)
    if "Living donor" in value:
        return 0
    else:
        return 1

def convert_age(value):
    """Placeholder function for age conversion - not used as age data is not available"""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion - not used as gender data is not available"""
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # We need to properly load the clinical data first from the matrix file
    try:
        # Assume clinical_data is already loaded from a previous step
        # If it's not available, we'll handle the exception
        if 'clinical_data' in locals() or 'clinical_data' in globals():
            # Use the geo_select_clinical_features function to extract clinical features
            clinical_features = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the extracted clinical features
            preview = preview_df(clinical_features)
            print("Preview of clinical features:")
            print(preview)
            
            # Save the clinical features to the specified output file
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            clinical_features.to_csv(out_clinical_data_file, index=False)
            print(f"Clinical features saved to {out_clinical_data_file}")
        else:
            print("Clinical data not available from previous step. Skipping clinical feature extraction.")
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")
        print("Skipping clinical feature extraction.")


### Step 3: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# Analyzing gene identifiers
# These identifiers have the format [number]_at
# This appears to be Affymetrix probe IDs, not standard human gene symbols
# The "_at" suffix is characteristic of Affymetrix microarray probe identifiers
# These will need to be mapped to standard gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Analyze the identifiers and determine mapping columns
# From previous steps, we know:
# - Gene expression data has identifiers like '100009613_at', which are Affymetrix probe IDs
# - Gene annotation data has 'ID' and 'ENTREZ_GENE_ID' columns

# The 'ID' column in gene_annotation contains the probe identifiers matching gene_data.index
# The 'ENTREZ_GENE_ID' column contains Entrez Gene IDs which are a type of gene identifier

# 2. Get gene mapping dataframe
probe_col = 'ID'
gene_col = 'ENTREZ_GENE_ID'
mapping_df = get_gene_mapping(gene_annotation, probe_col, gene_col)

print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("First 5 rows of mapping dataframe:")
print(mapping_df.head())

# Modify the apply_gene_mapping function for this specific case to use Entrez IDs directly
def custom_apply_gene_mapping(expression_df, mapping_df):
    """Modified version of apply_gene_mapping that doesn't use extract_human_gene_symbols
    but works directly with Entrez Gene IDs"""
    mapping_df = mapping_df[mapping_df['ID'].isin(expression_df.index)].copy()
    
    # Count genes per probe and expand to one gene per row
    mapping_df['num_genes'] = 1  # Each Entrez ID is one gene
    mapping_df.set_index('ID', inplace=True)

    # Merge and distribute expression values
    merged_df = mapping_df.join(expression_df)
    expr_cols = [col for col in merged_df.columns if col not in ['Gene', 'num_genes']]
    merged_df[expr_cols] = merged_df[expr_cols].div(merged_df['num_genes'], axis=0)

    # Sum expression values for each gene
    gene_expression_df = merged_df.groupby('Gene')[expr_cols].sum()

    return gene_expression_df

# 3. Convert probe-level measurements to gene expression data
mapped_gene_data = custom_apply_gene_mapping(gene_data, mapping_df)

print(f"\nGene expression data after mapping:")
print(f"Shape: {mapped_gene_data.shape}")
print("First 5 genes and 5 samples (if available):")
if not mapped_gene_data.empty and mapped_gene_data.shape[0] >= 5 and mapped_gene_data.shape[1] >= 5:
    print(mapped_gene_data.iloc[:5, :5])
else:
    print("Not enough data to display first 5x5. Sample data:")
    if not mapped_gene_data.empty:
        print(mapped_gene_data.iloc[:min(5, mapped_gene_data.shape[0]), 
                                :min(5, mapped_gene_data.shape[1])])

# Use the mapped data as our gene data
gene_data = mapped_gene_data

# Save the gene data to a file for future use
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the obtained gene expression data
try:
    # Now let's normalize the gene data using the provided function
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"First few gene symbols after normalization: {list(normalized_gene_data.index[:10])}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error in gene normalization: {e}")
    # If normalization fails, use the original gene data
    normalized_gene_data = gene_data
    print("Using original gene data without normalization")

# 2. Load the clinical data - make sure we have the correct format
try:
    # Load the clinical data we saved earlier to ensure correct format
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data:")
    print(clinical_data.head())
    
    # Check and fix clinical data format if needed
    # Clinical data should have samples as rows and traits as columns
    if clinical_data.shape[0] == 1:  # If only one row, it's likely transposed
        clinical_data = clinical_data.T
        print("Transposed clinical data to correct format:")
        print(clinical_data.head())
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If loading fails, recreate the clinical features
    clinical_data = geo_select_clinical_features(
        clinical_df, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    ).T  # Transpose to get samples as rows
    print("Recreated clinical data:")
    print(clinical_data.head())

# Ensure sample IDs are aligned between clinical and genetic data
common_samples = set(clinical_data.index).intersection(normalized_gene_data.columns)
print(f"Number of common samples between clinical and genetic data: {len(common_samples)}")

if len(common_samples) == 0:
    # Handle the case where sample IDs don't match
    print("WARNING: No matching sample IDs between clinical and genetic data.")
    print("Clinical data index:", clinical_data.index.tolist())
    print("Gene data columns:", list(normalized_gene_data.columns[:5]) + ["..."])
    
    # Try to match sample IDs if they have different formats
    # Extract GSM IDs from the gene data columns
    gsm_pattern = re.compile(r'GSM\d+')
    gene_samples = []
    for col in normalized_gene_data.columns:
        match = gsm_pattern.search(str(col))
        if match:
            gene_samples.append(match.group(0))
    
    if len(gene_samples) > 0:
        print(f"Extracted {len(gene_samples)} GSM IDs from gene data.")
        normalized_gene_data.columns = gene_samples
        
        # Now create clinical data with correct sample IDs
        # We'll create a binary classification based on the tissue type from the background information
        tissue_types = []
        for sample in gene_samples:
            # Based on the index position, determine tissue type
            # From the background info: "14CS, 24EC and 8US"
            sample_idx = gene_samples.index(sample)
            if sample_idx < 14:
                tissue_types.append(1)  # Carcinosarcoma (CS)
            else:
                tissue_types.append(0)  # Either EC or US
        
        clinical_data = pd.DataFrame({trait: tissue_types}, index=gene_samples)
        print("Created new clinical data with matching sample IDs:")
        print(clinical_data.head())

# 3. Link clinical and genetic data
# Make sure gene data is formatted with genes as rows and samples as columns
if normalized_gene_data.index.name != 'Gene':
    normalized_gene_data.index.name = 'Gene'

# Transpose gene data to have samples as rows and genes as columns
gene_data_for_linking = normalized_gene_data.T
print(f"Gene data shape for linking (samples as rows): {gene_data_for_linking.shape}")

# Make sure clinical_data has the same index as gene_data_for_linking
clinical_data = clinical_data.loc[clinical_data.index.isin(gene_data_for_linking.index)]
gene_data_for_linking = gene_data_for_linking.loc[gene_data_for_linking.index.isin(clinical_data.index)]

# Now link by concatenating horizontally
linked_data = pd.concat([clinical_data, gene_data_for_linking], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 columns):")
sample_cols = [trait] + list(linked_data.columns[1:5]) if len(linked_data.columns) > 5 else list(linked_data.columns)
print(linked_data[sample_cols].head())

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# Check if we still have data
if linked_data.shape[0] == 0 or linked_data.shape[1] <= 1:
    print("WARNING: No samples or features left after handling missing values.")
    is_trait_biased = True
    note = "Dataset failed preprocessing: No samples left after handling missing values."
else:
    # 5. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_trait_biased}")
    note = "This dataset contains gene expression data from uterine corpus tissues, comparing carcinosarcoma with endometrioid adenocarcinoma and sarcoma."

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")